In [1]:
from cassandra.cluster import Cluster
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')





from datetime import datetime
from datetime import timedelta
import pandas as pd
import time

In [ ]:
def StreamingStat2(session):
    #Process Remaining data
    #max_day = get the max day in stat2
    last_day = pd.DataFrame(list(session.execute('SELECT max(day) as last_day FROM "statistics_2";')))['day'][0]

    if last_day != None: #stat2 is not empty
        last_day = last_day.split(" ")[0]
        #get the the timestamp for the last processed record
        last_record_ts = pd.DataFrame(list(session.execute(f'SELECT max(LPRD) as last_record FROM "statistics_2" where \
            day=\'{last_day}\' allow filtering;')))['last_record'][0]
        #get the day that follow the current date 
        date_1 = datetime.strptime(last_day, "%Y-%m-%d")
        end_date = date_1 + timedelta(days=1)
        #get the data ordrer by timestamp
        data = pd.DataFrame(list(session.execute(f"SELECT * FROM Event where \
            timestamp > '{last_record_ts}' and  timestamp < '{end_date}' order by timestamp allow filtering;")))

        if(data.shape[0]>0):#if there is remaining data
            #get the hour of transaction for the first record

            data['from'] = data['timestamp'].apply(lambda x: (str(x).split(" ")[1]).split(":")[0])

            #get the first start_hour
            from_0 = data.loc[0,"from"]
            
            #check if the start_hour exists in statistics_2

            check_query = ""


In [ ]:
############ Process Remaining data
#->max_day = get the max day in stat2 
#->max_ts_day = max timestamp for the current day from stat2
#->data = data from event table where timestamp > max_ts_day and timestamp <next_day 
#->sort data by timestamp
#->start_hour = data[0]
#->i=0   old: 5,5,5  new: 5, 5, 5, 5, 6
#->if(start_hour already exists):#updating
#   ->stats = stats from stats2 where start_hour =new_data[start_hour]
#   ->while(i < len(data)):
#           ->sh = data[hour]
#           ->if(sh != start_hour):
#               ->new_hour = True
#               ->break
#           ->{users++, data.loc[i,start_station]++}
#           ->i++
#           ->if(i==len(data.shape[0]-1)):
#             
#                   ->update(id_event,date,start_hour,stats,data.loc[i,timestamp])
#   ->if(start_hour == True):
#   ->start_hour = data[0]
#   ->stats = {users=0}
#   ->while(i< len(data.shape[0])):
#       ->if(start_hour == z.loc[i,hour]): 
#           ->{users++, z.loc[i,start_station]++}
#           ->if(i==len(data)-1):
#               ->insert(id_event,date,start_hour,stats,z.loc[i,timestamp])
#           ->i++
#       ->else:
#           ->insert(id_event,date,start_hour,stats,z.loc[i,timestamp])
#           ->stats={users=0}
#           ->start_hour=z.loc[i,hour]
#
#
#
#
############### Process data from beginning
# ->for each date:
#   ->get data from event for the current date
#   ->sort data by timestamp
#   ->start_hour = data[0]
#   ->stats = {users=0}
#   ->i=0
#   ->while(i< len(data.shape[0])):
#       ->if(start_hour == z.loc[i,hour]): 
#           ->{users++, z.loc[i,start_station]++}
#           ->if(i==len(data)-1):
#               ->insert(id_event,date,start_hour,stats,data.loc[i,timestamp])
#           ->i++
#       ->else:
#           ->insert(id_event,date,start_hour,stats,data.loc[i,timestamp])
#           ->stats={users=0}
#           ->start_hour=data.loc[i,hour]
#
#############Process Streaming data
#
#->while(True):
#   ->sleep(5)
#   ->lts = get the max timestamp LSTP from stat2
#   ->data = check if there is new data in event where timestamp > lts
#   ->sort data by timestamp
#   ->start_hour = data[0]
#   ->i=0   old: 5,5,5  new: 5, 5, 5, 5, 6
#   ->if(start_hour already exists):#updating
#       ->stats = stats from stats2 where start_hour =new_data[start_hour]
#       ->while(i < len(data)):
#           ->sh = data[hour]
#           ->if(sh != start_hour):
#               ->new_hour = True
#               ->break
#           ->{users++, data.loc[i,start_station]++}
#           ->i++
#           ->if(i==len(data.shape[0]-1)):
#        
#                   ->update(id_event,date,start_hour,stats,data.loc[i,timestamp])
#       ->if(start_hour == True):
#       ->start_hour = data[i]
#       ->stats = {users=0}
#       ->while(i< len(data.shape[0])):
#           ->if(start_hour == z.loc[i,hour]): 
#               ->{users++, z.loc[i,start_station]++}
#               ->if(i==len(data)-1):
#                   ->insert(id_event,date,start_hour,stats,z.loc[i,timestamp])
#               ->i++
#       ->else:
#           ->insert(id_event,date,start_hour,stats,z.loc[i,timestamp])
#           ->stats={users=0}
#           ->start_hour=z.loc[i,hour]
#
#
#
#
#
#
#
#